In [27]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [33]:
def get_script(key,vote,type):

    dataset_name = os.path.join(key,vote)
    
    # script_path = f'train_script/{key}'

    # if not os.path.exists(script_path):
    #     os.mkdir(script_path)

    if type == 'train':
        script_name = f'scripts/train_{key}_{vote}.sh'

        with open (script_name,'w') as f:

            f.write('MODEL_FLAGS="--image_size 64 --num_channels 128 --num_res_blocks 3 --learn_sigma True --class_cond False"\n')
            f.write('DIFFUSION_FLAGS="--diffusion_steps 2000 --noise_schedule linear"\n')
            f.write('TRAIN_FLAGS="--lr 1e-4 --batch_size 5 "\n')
            f.write(f'DATA_DIR="--data_dir ..."\n')
            f.write(f'export OPENAI_LOGDIR=" ..."\n')
            f.write('mpiexec -n 8 python image_train.py $DATA_DIR $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS \n')
    else:
        script_name = f'scripts/sample_{key}_{vote}.sh'
        ema_path = []

        for path in os.listdir(f'your model path...'):
            if path.split('_')[0] == 'ema':
                ema_path.append(path)

        with open (script_name,'w') as f:

            f.write('MODEL_FLAGS="--image_size 64 --num_channels 128 --num_res_blocks 3 --learn_sigma True --class_cond False"\n')
            f.write('DIFFUSION_FLAGS="--diffusion_steps 2000 --noise_schedule linear"\n')
            f.write(f'MODEL_PATH="--model_path ......"\n')
            f.write(f'SAMPLE_FLAGS="--use_ddim True --timestep_respacing ddim250 --num_samples 128"\n')

            f.write(f'export OPENAI_LOGDIR="  "\n')
    
            f.write('mpiexec -n 8 python image_sample.py  $MODEL_FLAGS $DIFFUSION_FLAGS $MODEL_PATH $SAMPLE_FLAGS\n')

In [34]:
key = 'round'
vote = '0.9_0.99'
type = 'sample'

In [35]:
get_script(key,vote,type)

In [36]:
def get_screen(key,vote,type):

    sh_path = f'./screen_script/{key}'
    if not os.path.exists(sh_path):
        os.mkdir(sh_path)

    sh_name = f'{type}_{vote}.sh'

    with open(os.path.join(sh_path,sh_name),'w') as f:

        f.write(f'screen -dmS {type}_{key}\n')
        f.write('sleep 1\n')
        f.write(f"screen -r {type}_{key} -X stuff $'cd ./scripts\\n'\n")
        f.write(f"screen -r {type}_{key} -X stuff $'conda activate yolov5\\n'\n")
        f.write(f"screen -r {type}_{key} -X stuff $'bash {type}_{key}_{vote}.sh\\n'\n")

In [37]:
get_screen(key,vote,type)

In [14]:
!bash 'screen_script/{key}/{type}_{vote}.sh'